<a href="https://colab.research.google.com/github/benevolent-machines/crm-deduper/blob/main/docs/main-module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CRM Deduper Module

See our repository for more information:

https://github.com/benevolent-machines/crm-deduper


## _temp
A hidden global namespace for storing temporary copies of local variables that can be inspected outside of their scope.  

In [25]:
class _temp: pass

## _time()

A helper function that returns a string with the current time and elapsed time since the last call.

In [26]:
_last = None
def _time():
    from datetime import datetime
    global _last
    now = datetime.now()
    header = ""
    if _last is None:
        elapsed = "00:00.000"  # Initial call, no elapsed time
    else:
        delta = now - _last
        minutes, seconds = divmod(delta.total_seconds(), 60)
        milliseconds = delta.microseconds / 1000
        elapsed = f"{int(minutes):02}:{int(seconds):02}.{int(milliseconds):03}"
    _last = now
    f_time = now.strftime('%M:%S.%f')[:-3]
    return f"{f_time} {elapsed}"

In [27]:
#test
if False:
    import time
    print(_time(),'first')  # First call, elapsed will be "00:00.000"
    time.sleep(1.234)  # Sleep for a bit over a second to see the change
    print(_time(), 'next')  # Second call, should show elapsed time since first call

## _similar()
A helper function to quickly assess the similarity between two strings, useful for refining machine learning recall into higher precision levels. This Python code was adapted from a T-SQL function found here:

https://www.sqlservercentral.com/articles/fuzzy-search

In [28]:
def _similar(s1, s2):
    if len(s1) > len(s2):
        (s1, s2) = (s2, s1) # easy switch in python!
    k = len(s1)
    if k < 2:
        return 0.0
    else:
        j = 0
        for i in range(len(s1) - 1):
            substring = s1[i:i+2]
            if substring in s2:
                j += 1
        return (j / (k-1)) * 100.0

In [29]:
#test
if False:
    print(f"{_similar('roger','susan')}")
    print(f"{_similar('example','example')}")

## _load_data()
A helper function to load data from either a csv or excel file.

In [30]:
def _load_data(input_data):
    import pandas as pd

    if isinstance(input_data, str):
        file_extension = input_data.split('.')[-1].lower()
        if file_extension == 'csv':
            df = pd.read_csv(input_data)
        elif file_extension in ['xlsx', 'xls']:
            df = pd.read_excel(input_data)
        else:
            raise ValueError(
                f"load_data(): Unsupported file type: {file_extension}")
    elif isinstance(input_data, pd.DataFrame):
        df = input_data
    else:
        raise ValueError(
            f"load_data(): Unsupported file type: {file_extension}")

    string_columns = df.select_dtypes(include=['object']).columns
    df[string_columns] = df[string_columns].fillna('')
    return df

#_load_data("test.csv")

## _regroup_pass()
A function designed to regroup rows during a deduplication pass for more even distribution, based on a specified threshold.

In [31]:
def _regroup_pass(df, col, num, threshold=2000):

    # Slice the first `num` characters of `col` for grouping
    sliced_col = df[col].str[:num].str.upper()

    # Read sliced_col counts in descending order
    col_counts = sliced_col.value_counts().sort_values(ascending=False)
    accumulated = 0
    group_id = 1  # Start sequence number for small groups
    group_mapping = {}

    for col_name, count in col_counts.items():
        if count >= threshold:
            # Categories larger than the threshold keep their original name
            group_mapping[col_name] = col_name
        else:
            # Accumulate small categories until they meet the threshold
            if accumulated + count <= threshold:
                group_mapping[col_name] = f'_{group_id}'
                accumulated += count
            else:
                # Once the threshold is exceeded, start a new group
                group_id += 1
                group_mapping[col_name] = f'_{group_id}'
                accumulated = count  # Reset accumulated count for the new group

    # Apply the group mapping based on the sliced_col values
    df['group'] = sliced_col.map(group_mapping)
    return df

In [32]:
#test
if False:
    df = _regroup_pass(df, 'state', 2, 3000)
    group_sizes = df.groupby('group').size()
    # Display count, maximum, and average of group sizes
    print(f"Total Groups: {group_sizes.count()}")
    print(f"Maximum Group Size: {group_sizes.max()}")
    print(f"Median Group Size: {group_sizes.median()}")
    df['group'].value_counts().sort_values(ascending=False)

## _find_duplicates()
The main algorithm function for finding duplicates in large datasets.

In [33]:
def _find_duplicates(
    pandas_dataframe, # required
    entity_id = 'entity_id',
    search_columns = ['name', 'address', 'city', 'state'],
    dedupe_passes = ['ALL'],  # ['state:2', 'name:3']
    similar_column = None,
    score_minimum = 0.7,
):
    import numpy as np
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.neighbors import NearestNeighbors

    df = pandas_dataframe

    if dedupe_passes is None:
        dedupe_passes = ['ALL']
    elif isinstance(dedupe_passes, str):
        dedupe_passes = [dedupe_passes]
    elif not isinstance(dedupe_passes, list):
        raise ValueError("invalid dedupe_passes parameter")

    pass_df_list = []
    df['pass'] = None
    df['group'] = None # initialize to avoid pandas warnings

    for dedupe_pass in dedupe_passes: # ['state:2', 'city'] or 'ALL'
        print(f"{_time()} pass: {dedupe_pass}")
        pass_list = dedupe_pass.split(':')
        pass_col = pass_list[0]
        pass_len = int(pass_list[1]) if len(pass_list) > 1 else 2
        df['pass'] = dedupe_pass
        if pass_col == 'ALL':
            df['group'] = 'ALL'
        else:
            df[pass_col] = df[pass_col].astype(str)
            df = _regroup_pass(df, pass_col, pass_len) # updates df['group']

        gdf_list = []

        vectorizer = TfidfVectorizer(
            stop_words=None,
            ngram_range=(1, 1), min_df=2, max_df=0.8)

        for group, gdf in df.groupby('group'):
            print(f"{_time()} group: {group} shape[0]: {gdf.shape[0]}")
            gdf = gdf.sort_values(entity_id, ascending=False)
            gdf = gdf.reset_index(drop=True)
            gdf[search_columns] = gdf[search_columns].astype(str)
            gdf['combined'] = gdf[search_columns].agg(' '.join, axis=1)
            gdf['combined'] = gdf['combined'].str.upper()
            gdf['last_id'] = gdf[entity_id]
            gdf['dup_flag'] = 0
            gdf['score'] = 0.0

            tfidf_matrix = vectorizer.fit_transform(gdf['combined'])
            print(f"{_time()} vectorizer.fit_transform() completed")

            nbrs = NearestNeighbors(
                radius=0.33, metric='cosine', algorithm='brute')
            print(f"{_time()} NearestNeighbors() completed")

            nbrs.fit(tfidf_matrix)
            print(f"{_time()} nbrs.fit() completed")

            distances, indices = nbrs.radius_neighbors(tfidf_matrix)
            print(f"{_time()} nbrs.radius_neighbors() completed")

            for i, (dist, idx) in enumerate(zip(distances, indices)):
                for d, j in zip(dist, idx):
                    if i < j:  # only compare with what comes after
                        if (similar_column is not None
                            and _similar(gdf.loc[i, similar_column],
                                         gdf.loc[j, similar_column]) < 50
                        ):
                            continue; # not similar
                        score = 1 - d  # Convert distance to similarity
                        if score > score_minimum:
                            if gdf.loc[i, 'dup_flag'] == 0:
                                gdf.loc[i, 'dup_flag'] = 2
                            gdf.loc[j, 'dup_flag'] = 1
                            if gdf.loc[j, 'last_id'] == gdf.loc[j, entity_id]:
                                gdf.loc[j, 'last_id'] = gdf.loc[i, entity_id]
                                if gdf.loc[i, 'score'] == 0.0:
                                    gdf.loc[i, 'score'] = 1.0
                                gdf.loc[j, 'score'] = round(score, 3)

            gdf_list.append(gdf)

        pass_df = pd.concat(gdf_list, ignore_index=True)
        #pass_df.drop(columns=['group'], inplace=True)

        pass_df_list.append(pass_df[pass_df['dup_flag'].isin([1,2])]) # dups
        df = pass_df[pass_df['dup_flag'] == 0].copy() # drop dups in next pass

    results_df = pd.concat(pass_df_list, ignore_index=True)
    #results_df.drop(columns=['group'], inplace=True)
    #results_df = results_df.copy()
    #m.df = df.copy()
    all_df = pd.concat([df, results_df], ignore_index=True)
    #all_df.to_csv("labeled_out.csv")

    _temp.audit_df = all_df

    return all_df

## _excel_redirect()
A helper function to ensure that links in Excel can access CRM functions without being sent to a login screen each time.

In [34]:
def _excel_redirect(url):
    redirector = "https://benevolentmachines.org/excel_redirect.html"
    url = redirector + '?page=' + url.replace('?',"&")
    return url

## _link_templates
A dictionary of dictionaries with templates used to construct links in the Excel output file.  

In [35]:
_link_templates = {
    "donorperfect": {
        "record": "https://www.donorperfect.net/prod/donor.asp"
            + "?donor_id={entity_id}",
        "merge": "https://www.donorperfect.net/prod/combinedonorsStart.asp"
            + "?donor_id1={entity_id}&donor_id2={last_id}&combineType=Advanced"
    },
}

## _output_excel()
The function used to output found duplicates in a formatted Excel document.  

In [36]:
#test - The invoking context must install this package and datamachine to work.
!pip install XlsxWriter

In [37]:
def _output_excel(
    pandas_dataframe,
    output_file='output.xlsx',
    entity_id = 'entity_id',
    search_columns = ['name', 'address', 'city', 'state'],
    link_templates_key = None,
    output_all = False, # True outputs all records, False is just the dups
):
    import pandas as pd
    import xlsxwriter

    # organize the columns
    xdf = pandas_dataframe
    xdf['type'] = xdf['dup_flag']
    initial = ['type', entity_id, 'last_id', 'score']
    reference = []  # additional fields
    for col in xdf.columns: # extract references for the spreadsheet
        if col not in initial and col not in search_columns:
            if col != 'rownum' and col != 'combined':
                reference.append(col)
    ordered = initial + search_columns + reference
    if link_templates_key is not None:
        link_templates_key = link_templates_key.lower()

    # prepare the dataframe for the spreadsheet
    if output_all == False:
        xdf = xdf[xdf['dup_flag'] > 0] # the spreadsheet only includes duplicates
    xdf = xdf[ordered] # the order that will appear in the spreadsheet
    xdf = xdf.sort_values(['last_id','dup_flag'], ascending=[False, False])
    xdf = xdf.reset_index(drop=True) # renumber after sorting
    xdf = xdf.fillna('') # important
    #xdf.rename(columns={'dup_flag': 'type'}, inplace=True)
    xdf['type'] = xdf['type'].map({2: 'last', 1: 'prior'}).astype(str)
    xdf['type'] = xdf['type'].str.replace('nan','')

    if link_templates_key is not None:
        xdf.insert(loc=3, column='action', value='')

    excel_file = output_file # 'output.xlsx'
    writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')

    xdf.to_excel(writer, index=False, sheet_name='Sheet1', startrow=1, header=False)
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Custom formats
    header_grey_left = workbook.add_format(
        {'bg_color': '#D3D3D3', 'bold': True, 'align': 'left'})
    header_grey_right = workbook.add_format(
        {'bg_color': '#D3D3D3', 'bold': True, 'align': 'right'})
    header_green_left = workbook.add_format(
        {'bg_color': '#B8D7A3', 'bold': True, 'align': 'left'})

    last_blue_left = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'left'})
    last_blue_right = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right'})
    last_blue_right_score = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right', 'num_format': '0.000'})
    last_blue_right_date = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right', 'num_format': 'yyyy-mm-dd'})
    last_blue_right_dec2 = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right', 'num_format': '#,##0.00'})
    last_blue_right_dec3 = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right', 'num_format': '##0.000'})
    last_blue_left_link = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'left',
        'font_color': 'blue', 'underline': 1})
    last_blue_right_link = workbook.add_format(
        {'bg_color': '#ADD8E6', 'align': 'right',
        'font_color': 'blue', 'underline': 1})

    prior_left = workbook.add_format({'align': 'left'})
    prior_right = workbook.add_format({'align': 'right'})
    prior_right_score = workbook.add_format(
        {'align': 'right', 'num_format': '0.000'})
    prior_right_date = workbook.add_format(
        {'align': 'right', 'num_format': 'yyyy-mm-dd'})
    prior_right_dec2 = workbook.add_format(
        {'align': 'right', 'num_format': '#,##0.00'})
    prior_right_dec3 = workbook.add_format(
        {'align': 'right', 'num_format': '##0.000'}) # score
    prior_left_link = workbook.add_format(
        {'align': 'left', 'font_color': 'blue', 'underline': 1})
    prior_right_link = workbook.add_format(
        {'align': 'right', 'font_color': 'blue', 'underline': 1})

    ##### determine column properties
    tdf = xdf[0:500] # look at first n rows to extract properties
    col_info = {} # info about the columns
    for col in tdf.columns:
        col_props = {}
        is_numeric = is_string = is_date = False
        max_decimals = 0
        is_numeric = pd.to_numeric(
            tdf[col].dropna(), errors='coerce').notnull().all()
        if is_numeric:
            numeric_series = pd.to_numeric(tdf[col], errors='coerce').dropna()
            decimal_lengths = numeric_series.apply(
                lambda x: len(str(x).split('.')[1]) if '.' in str(x) else 0)
            max_decimals = decimal_lengths.max()
        else:
            is_date = pd.to_datetime(
                tdf[col].dropna(), errors='coerce').notnull().all()
            if not is_date:
                is_string = tdf[col].dtype == 'object'
        col_props['is_numeric'] = is_numeric
        col_props['is_string'] = is_string
        col_props['is_date'] = is_date
        col_props['max_decimals'] = max_decimals
        col_info[col] = col_props

    # column settings
    for num, col in enumerate(tdf.columns):

        # Calculate the maximum length of data in the column
        if tdf.empty:
            max_data_len = 0
        else:
            max_data_len = tdf[col].astype(str).map(len).max()

        # Consider the length of the column name as well
        col_name_len = len(col)

        # Determine the final column width
        max_len = max(max_data_len, col_name_len) + 1  # +1 little extra
        #print(col, max_len)
        if max_len > 15:
            max_len = 15
        worksheet.set_column(num, num, max_len)

        # custom headers
        if col in search_columns: # comparison columns get a green header
            worksheet.write(0, num, col, header_green_left)
        else:
            if col == 'type':
                worksheet.write(0, num, col, header_grey_right)
            elif col_info[col]['is_string']:
                worksheet.write(0, num, col, header_grey_left)
            else:
                worksheet.write(0, num, col, header_grey_right)

    # cell settings
    for idx, row in xdf.iterrows():
        row_num = idx + 1  # account for header row
        for col_num, col in enumerate(tdf.columns):
            if row['type'] == 'last':
                if col == 'type':
                    worksheet.write(row_num, col_num, row[col],
                                    last_blue_right)
                elif col == entity_id:
                    if link_templates_key is not None: # hyperlinks
                        url = _link_templates[
                            link_templates_key]["record"].format(
                                entity_id = row[entity_id]
                            )
                        url = _excel_redirect(url) # excel nonsense
                        if col_info[col]["is_string"]:
                            worksheet.write_url(row_num, col_num, url,
                                last_blue_left_link,
                                string=str(row[entity_id]))
                        else:
                            worksheet.write_url(row_num, col_num, url,
                                last_blue_right_link,
                                string=str(row[entity_id]))
                    else:
                        if col_info[col]["is_string"]:
                            worksheet.write(
                                row_num, col_num, row[col], last_blue_left)
                        else:
                            worksheet.write(
                                row_num, col_num, row[col], last_blue_right)
                elif col == 'action': # blank and blue
                    worksheet.write(row_num, col_num, '', last_blue_right)
                elif col == 'score': # blank and blue
                    worksheet.write(row_num, col_num, '', last_blue_right)
                else:
                    if col_info[col]["is_string"]:
                        worksheet.write(row_num, col_num, row[col],
                                        last_blue_left)
                    elif col_info[col]['is_numeric']:
                        if col_info[col]['max_decimals'] == 2:
                            worksheet.write(row_num, col_num, row[col],
                                            last_blue_right_dec2)
                        else:
                            worksheet.write(row_num, col_num, row[col],
                                            last_blue_right)
                    elif col_info[col]['is_date']:
                        worksheet.write(row_num, col_num, row[col],
                                        last_blue_right_date)
                    else: # other?
                        worksheet.write(row_num, col_num, row[col],
                                        last_blue_right)
            else: # prior
                if col == 'type':
                    worksheet.write(row_num, col_num, row[col], prior_right)
                elif col == entity_id:
                    if link_templates_key is not None: # hyperlinks
                        url = _link_templates[
                            link_templates_key]["record"].format(
                                entity_id = row[entity_id]
                            )
                        url = _excel_redirect(url) # excel nonsense
                        if col_info[col]["is_string"]:
                            worksheet.write_url(row_num, col_num, url,
                                prior_left_link, string=str(row[entity_id]))
                        else:
                            worksheet.write_url(row_num, col_num, url,
                                prior_right_link, string=str(row[entity_id]))
                    else:
                        if col_info[col]["is_string"]:
                            worksheet.write(
                                row_num, col_num, row[col], prior_left)
                        else:
                            worksheet.write(
                                row_num, col_num, row[col], prior_right)
                elif col == 'action':
                    if link_templates_key is not None:
                        url = _link_templates[
                            link_templates_key]["merge"].format(
                                entity_id = row[entity_id],
                                last_id = row["last_id"]
                            )
                        url = _excel_redirect(url) # excel nonsense
                        worksheet.write_url(row_num, col_num, url,
                            prior_left_link, string='merge')
                elif col == 'score':
                    worksheet.write(row_num, col_num, row[col],
                                    prior_right_dec3)
                else:
                    if col_info[col]["is_string"]:
                        #print(col)
                        worksheet.write(row_num, col_num, row[col],
                                        prior_left)
                    elif col_info[col]['is_numeric']:
                        if col_info[col]['max_decimals'] == 2:
                            worksheet.write(row_num, col_num, row[col],
                                            prior_right_dec2)
                        else:
                            worksheet.write(row_num, col_num, row[col],
                                            prior_right)
                    elif col_info[col]['is_date']:
                        worksheet.write(row_num, col_num, row[col],
                                        prior_right_date)
                    else: # other?
                        worksheet.write(row_num, col_num, row[col],
                                        prior_right)

    worksheet.freeze_panes(1, 4)
    writer.close()
    return xdf

## run()

In [38]:
def run(
    input_data, # file (xls,xlsx,csv) or dataframe
    entity_id = 'entity_id', # unique id needed for linking
    search_columns = ['first_name', 'last_name', 'address', 'city', 'state'],
    similar_column = None, # 'first_name' could exclude household matches
    amount_column = None, # 'total_amount', for instance
    amount_minimum = None, # the minimum amount to isolate major pairs
    recent_column = None, # 'insert_date' could be used for recent records
    recent_days = None, # the number of recent days with recent_column
    string_column = None, # can be 'created_by' for source
    string_equals = None, # the value used with string_column
    exclude_previous = False, # uses previous.xlsx to exclude matches
    link_templates_key = None, # templates for CRM record and merge links
    # the following parameters are experimental
    output_all = False, # True outputs all records, False is just the dups
    score_minimum = .67,
    dedupe_passes = ['ALL'],  # ['state:2', 'name:3']
    tuning = False,
    quiet = False,
    limit = None,
):
    """
    Runs deduplication on address data based on configurable parameters.

    Documentation is available at:
    https://github.com/benevolent-machines/crm-deduper

    """
    global _last
    _last = None
    # suppress warning
    import warnings
    import dateutil
    import os
    import shutil
    import pandas as pd

    warnings.filterwarnings("ignore",
        category=dateutil.parser.UnknownTimezoneWarning)

    if not quiet: print(f"{_time()} loading data")
    df = _load_data(input_data) # load file
    if limit is not None:
        df = pd.DataFrame(df[0:limit])

    if not quiet: print(f"{_time()} finding duplicates")
    df = _find_duplicates(
        df,
        entity_id = entity_id,
        search_columns = search_columns,
        dedupe_passes = dedupe_passes,
        score_minimum = .67,
        similar_column = similar_column,
    )

    # limit the output to pairs including the largest amounts
    if amount_column is not None and amount_minimum is not None:
        # filter on duplications involving amount_field at a amount_minimum
        last_ids = list(
            df[df[amount_column] >= amount_minimum]['last_id'].unique())
        df = df[df['last_id'].isin(last_ids)].copy()

    # limit the output to pairs with recent activity
    if recent_column is not None and recent_days is not None:
        df[recent_column] = pd.to_datetime(df[recent_column]) # convert
        cutoff_date = pd.Timestamp.now() - pd.Timedelta(days=recent_days)
        last_ids = list(
            df[df[recent_column] >= cutoff_date]['last_id'].unique())
        df = df[df['last_id'].isin(last_ids)].copy()

    # limit the output to pairs including a specific string value
    if string_column is not None and string_equals is not None:
        last_ids = list(
            df[df[string_column] >= string_equals]['last_id'].unique())
        df = df[df['last_id'].isin(last_ids)].copy()

    if tuning:
        return df # run results for tuning and avoid excel outputs
    else:
        if exclude_previous:
            if os.path.isfile('previous.xlsx') and not output_all:
                prev_df = pd.read_excel('previous.xlsx')
                c1 = ~df[entity_id].isin(prev_df[entity_id])
                df = df[c1].copy() # remove previous

        if not quiet: print(f"{_time()} outputting Excel")
        xdf = _output_excel(
            df,
            entity_id=entity_id,
            search_columns=search_columns,
            link_templates_key=link_templates_key,
            output_all=output_all,
        )
        if not quiet: print(f"{_time()} results completed in output.xlsx")

        if exclude_previous and os.path.isfile('previous.xlsx'):
            prev_df.drop(['action', 'type'], axis=1, inplace=True)
            c1 = df['dup_flag'].isin([1,2]) # duplicates
            prev_df = pd.concat([df[c1], prev_df]).copy() # new previous
            xprev_df = _output_excel(
                prev_df, output_file='previous.xlsx',
                entity_id=entity_id,
                search_columns=search_columns,
                link_templates_key=link_templates_key,
                output_all=output_all,
            )
        else: # initialize the previous as the output
            shutil.copy('output.xlsx','previous.xlsx')

        return xdf # formatted results for users - also in output.xlsx

In [39]:
#module_end  datamachine end of module

# TESTS

## CRM Data (CSV)

In [40]:
if True:
    df = run(
        input_data = "donors.csv", # exported from the CRM
        entity_id = "donor_id", # unique id
        search_columns = ['first_name', 'last_name', 'address', 'city', 'state'],
        dedupe_passes = ['ALL'],
        link_templates_key = "donorperfect",
        similar_column = 'first_name',
        #recent_column = 'last_contrib_date',
        #recent_days = 7,
        amount_column = 'gift_total',
        amount_minimum = 1_000,
        output_all = False,
        tuning = False,
        quiet = False,
        #limit = 10_000,
    )

21:10.936 00:00.000 loading data
21:11.280 00:00.344 finding duplicates
21:12.809 00:01.528 pass: ALL
21:12.846 00:00.036 group: ALL shape[0]: 75232
21:14.013 00:01.166 vectorizer.fit_transform() completed
21:14.013 00:00.000 NearestNeighbors() completed
21:14.016 00:00.002 nbrs.fit() completed
22:46.176 01:32.160 nbrs.radius_neighbors() completed
22:46.945 00:00.768 outputting Excel
22:47.105 00:00.159 results completed in output.xlsx


# TUNING

## _get_metrics()

In [ ]:
def _get_metrics(df, quiet=False):
    import pandas as pd
    audit_df = df
    audit_df['dup_flag'] = audit_df['dup_flag'].fillna(0)
    audit_df['label'] = audit_df['label'].fillna(0)
    # First, let's define what constitutes a true positive, false positive, and false negative in your context
    # True Positive (TP): dup_flag is 1 or 2 AND label is 1 or 2
    # False Positive (FP): dup_flag is 1 or 2 BUT label is NOT 1 or 2
    # False Negative (FN): dup_flag is NOT 1 or 2 BUT label is 1 or 2
    total = audit_df.shape[0]
    labeled = audit_df[audit_df['label'].isin([1, 2])].shape[0]
    found = audit_df[audit_df['dup_flag'].isin([1, 2])].shape[0]
    TP = audit_df[(audit_df['dup_flag'].isin([1, 2])) & (audit_df['label'].isin([1, 2]))].shape[0]
    FP = audit_df[(audit_df['dup_flag'].isin([1, 2])) & (~audit_df['label'].isin([1, 2]))].shape[0]
    FN = audit_df[(~audit_df['dup_flag'].isin([1, 2])) & (audit_df['label'].isin([1, 2]))].shape[0]
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    if not quiet:
        print(f'total: {total}, labeled: {labeled}, found {found}')
        print(f"Precision: {precision*100:.2f}% : Of those found, the percent that were labeled")
        print(f"   Recall: {recall*100:.2f}% : Of those labeled, the percent that were found.")
    return (total, labeled, found, precision, recall)
#_get_metrics(df)

## tuning()

In [ ]:
if False:
    import pandas as pd
    from io import StringIO
    import ast
    import time
    zdf = pd.read_csv(StringIO( """\
    secs|precision|recall|file|passes
    0|0.0|0.0|labels2.csv|['first_name:2']
    0|0.0|0.0|labels2.csv|['last_name:2']
    0|0.0|0.0|labels2.csv|['last_name:2']
    0|0.0|0.0|labels2.csv|['state:2']
    0|0.0|0.0|labels2.csv|['ALL']
    0|0.0|0.0|labels2.csv|['state:2','ALL']
    """), delimiter='|',
    dtype={})
    for index, row in zdf.iterrows():
        start_time = round(time.time())

        print(f'test:{index} file:{row["file"]} passes:{row["passes"]}')
        passes = ast.literal_eval(row['passes'])
        df = run(
            input_data = row['file'], # exported from the CRM
            entity_id = "donor_id", # unique id
            search_columns = ['first_name', 'last_name', 'address', 'city', 'state'],
            dedupe_passes = passes,
            link_templates = "donorperfect",
            output_all = True, # testing
            tuning = True,
            quiet = True,
        )
        (total, labeled, found, precision, recall) = _get_metrics(df, quiet=True)
        zdf.at[index, 'precision'] = precision
        zdf.at[index, 'recall'] = recall
        zdf.at[index, 'secs'] = round(time.time()) - start_time
    zdf